|Column|Description|
|:------:|:---:|
|<code>category</code>| Topic (target variable) |
|<code>description</code>| Description |
|<code>text_content</code>| Text content |
|<code>title</code>| Title |
|<code>h1</code>| Content of the <code>h1</code> tag on the page |
|<code>h2</code>| Content of the <code>h2</code> tag on the page |
|<code>url</code>| Link address|
|<code>domain</code>| Website domain |
|<code>id</code>| Link ID|

In [1]:
import numpy as np
import pandas as pd
from hazm import Normalizer, word_tokenize
import string
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import  train_test_split
import seaborn as sns

In [2]:
df = pd.read_csv('./data/yektanet_train.csv')
df.head(1)

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158


# preprocessing

In [3]:
df['category'].value_counts()

category
سلامت                  614
ورزش                   514
حقوق و دولت و سیاست    486
هنر و سرگرمی           410
موسیقی                 314
تکنولوژی و کامپبوتر    287
تجارت و اقتصاد         283
فیلم و سینما           239
خودرو                  237
اجتماعی                209
سفر و گردشگری          182
غذا و نوشیدنی          171
مذهبی                  160
مسکن                   131
خانه و باغبانی         128
مد و زیبایی            118
کتاب و ادبیات           83
تحصیلات                 79
اشتغال                  47
علم و دانش              34
خانواده                 34
حیوانات خانگی           29
Name: count, dtype: int64

In [4]:
X_train = df.drop(columns=['category'])
y_train = df['category']

In [5]:
ros = RandomOverSampler(random_state=0)
data_resampled, targets_resampled = ros.fit_resample(X_train, y_train)

In [6]:
targets_resampled.value_counts()

category
کتاب و ادبیات          614
تجارت و اقتصاد         614
اشتغال                 614
خانواده                614
علم و دانش             614
تحصیلات                614
موسیقی                 614
خانه و باغبانی         614
هنر و سرگرمی           614
سفر و گردشگری          614
فیلم و سینما           614
مذهبی                  614
اجتماعی                614
مد و زیبایی            614
خودرو                  614
مسکن                   614
حقوق و دولت و سیاست    614
غذا و نوشیدنی          614
ورزش                   614
تکنولوژی و کامپبوتر    614
سلامت                  614
حیوانات خانگی          614
Name: count, dtype: int64

In [7]:
persian_stopwords = set([
    "از", "به", "در", "با", "که", "را", "تا", "و", "یا", "اما", "اگر", "برای", "بر", 
    "این", "آن", "یک", "هر", "هم", "همه", "چند", "چنین", "دیگر", "چون", "مثل", 
    "مانند", "چرا", "زیرا", "ولی", "آیا", "اگرچه", "لذا", "نیز", "باید", "می", 
    "باشد", "است", "بود", "هست", "شد", "شو", "باش", "کرد", "کن", "کند", "کرده", 
    "شده", "می‌شود", "خواهد", "خواهند", "خواهی", "خواهیم", "توان", "تواند", 
    "توانند", "توانست", "توانسته", "بوده", "نبود", "نباشد", "نیست", "نیستند", 
    "بودند", "باشند", "هستند", "دارم", "داری", "دارد", "دارند", "داریم", "داشت", 
    "داشتند", "داشته", "داشتم", "ندارم", "ندارد", "ندارند", "نداریم", "نداشت", 
    "نداشتند", "نداشته", "ای", "ایم", "اید", "اند", "ام", "ت", "ها", "های", "هایی", 
    "شان", "ش", "مان", "تان", "اینها", "آنها", "چیز", "چیزی", "چرا", "چه", "که", 
    "کدام", "چگونه", "چقدر", "چراکه", "آنان", "او", "آن", "ایشان", "ما", "شما", 
    "آنچه", "آنجا", "اینجا", "اینجاست", "آنجاست", "همان", "خود", "همه‌اش", 
    "هیچ", "هیچ‌کدام", "هرگز", "هیچگاه", "حالا", "اکنون", "دیروز", "امروز", 
    "فردا", "شب", "روز", "بعد", "قبل", "ساعت", "وقت", "زمان", "چندین", "بار", 
    "کم", "بیشتر", "کمتر", "حتی", "فقط", "تنها", "بالا", "پایین", "روی", "زیر", 
    "جلو", "پشت", "نزدیک", "دور", "وسط", "بیرون", "درون", "داخل", "کنار", 
    "اینجا", "آنجا", "هیچ‌جا", "هرجا", "هرکجا", "جا", "مکان", "محل", "چپ", "راست", 
    "بعدا", "سپس", "آنگاه", "دیگر", "چیزهای", "یعنی", "خب", "آره", "نه", "باشه", 
    "آها", "بله", "نمیدانم", "کسی", "دیگری", "هیچ‌کسی", "چیزها"
])

In [8]:
normalizer = Normalizer()

def preprocessor(input):
    punc_removed = input.translate(str.maketrans('', '', string.punctuation))
    normalized = normalizer.normalize(punc_removed)
    tokens = word_tokenize(normalized)
    filtered = []
    for token in tokens:
        token = str(token)
        token = token.lower()
        if not token in persian_stopwords and not token.isdigit():
            filtered.append(token)
    output = ' '.join(filtered)
    return output

In [9]:
def tokenizer(text):
    return word_tokenize(text)

In [10]:
encoder = LabelEncoder()
targets_resampled = encoder.fit_transform(targets_resampled)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data_resampled['title'], targets_resampled, test_size=0.2)

In [12]:
y_train

array([17, 16, 15, ..., 13, 10,  1])

In [17]:
svm = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor,
                                         analyzer='word', ngram_range=(1, 2),
                                         min_df=5, lowercase=False)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf-svm', LinearSVC(loss='hinge', penalty='l2', max_iter=5000))])

model = svm.fit(X_train, y_train)

e:\Courses&Code\ML_excersice\env\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [18]:
print('Training F1-score:', f1_score(y_train, model.predict(X_train), average='weighted'))
print('Test F1-score:', f1_score(y_test, model.predict(X_test), average='weighted'))

Training F1-score: 0.9786719365950901
Test F1-score: 0.9207918952208628


In [19]:
xgb = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor,
                                         analyzer='word', ngram_range=(1, 2),
                                         min_df=5, lowercase=False)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf-svm', XGBClassifier(  # XGBClassifier for multi-class classification
                                objective='multi:softmax',  # Use 'multi:softprob' for probabilities
                                num_class=22,  # Number of classes
                                eval_metric='mlogloss',  # Metric for multi-class classification
                                use_label_encoder=False,  # Disable label encoder
                                random_state=42
                            ))])

model = xgb.fit(X_train, y_train)

e:\Courses&Code\ML_excersice\env\lib\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
e:\Courses&Code\ML_excersice\env\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
print('Training F1-score:', f1_score(y_train, model.predict(X_train), average='weighted'))
print('Test F1-score:', f1_score(y_test, model.predict(X_test), average='weighted'))

Training F1-score: 0.9662009710899965
Test F1-score: 0.9120089016281424
